In [13]:
!pip install pandas 
!pip install lxml

   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   -------- ------------------------------- 0.8/3.8 MB 4.8 MB/s eta 0:00:01
   ---------- ----------------------------- 1.0/3.8 MB 5.0 MB/s eta 0:00:01
   ---------- ----------------------------- 1.0/3.8 MB 5.0 MB/s eta 0:00:01
   --------------------- ------------------ 2.1/3.8 MB 2.5 MB/s eta 0:00:01
   -------------------------------- ------- 3.1/3.8 MB 3.1 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 3.4 MB/s eta 0:00:00


In [5]:
import pandas as pd

In [ ]:
path = r"C:\Users\menga\Documents\Code\LLM-TextAnalyzer\datasets\result.xml"

In [15]:
print(path)

/datasets/result.xml


Anteprima DataFrame:


,xml/results/result/answers/answer/text#text,xml/results/result/answers/answer@id,xml/results/result@content_id
0,no,NaN,NaN
1,NaN,5dbd3c18-b7ac-4dfd-accb-97294f731bea,NaN
2,"Yes, the text mentions the development of a di...",NaN,NaN
3,NaN,0832d05a-662c-49c0-8bbf-4a950fe9787c,NaN
4,NaN,NaN,044897fa-5520-4e41-8a61-dd9a89f91ba8
5,no,NaN,NaN
6,NaN,5dbd3c18-b7ac-4dfd-accb-97294f731bea,NaN
7,"Yes, the text describes the implementation and...",NaN,NaN
8,NaN,0832d05a-662c-49c0-8bbf-4a950fe9787c,NaN
9,NaN,NaN,26b70aa6-0a69-4058-b58f-9156402386d5
